# LSTM for puzzle allocation

In [1]:
import numpy as np
import tensorflow as tf 

2023-08-24 15:12:36.868785: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/glenda/opt/anaconda3/envs/machine-learning/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Attention spans in children
2 years old: four to six minutes <br>
4 years old: eight to 12 minutes  <br>
6 years old: 12 to 18 minutes <br>
8 years old: 16 to 24 minutes <br>
10 years old: 20 to 30 minutes <br>
12 years old: 24 to 36 minutes <br>

In [2]:
AGE_TARGET_TIME = {
  2: 300,
  3: 450,
  4: 600,
  5: 750,
  6: 900,
  7: 1050,
  8: 1200,
  9: 1350,
  10: 1500,
  11: 1650, 
  12: 1800,
}

AGE = 5

TARGET_TIME_IN_SECONDS = AGE_TARGET_TIME[AGE]

PROGRESSION_COEFF = 1.5 # relationship between rating and time taken to solve
PERF_QUOTIENT_NORMALISATION_COEFF = 5 # attempt to normalise the curve of the performance quotient

In [3]:
print(f"Target time in seconds: {TARGET_TIME_IN_SECONDS}s")

Target time in seconds: 750s


In [4]:
# Between -1 to 1. Quotient of 0 represents normal performance, lower number means poorer performance
# Approximate performanceQuotient to be normally distributed, with a mean of 0 
def performanceQuotient(userRating, puzzleRating, timeTaken):
  targetTime = (puzzleRating/userRating) * TARGET_TIME_IN_SECONDS * PROGRESSION_COEFF
  
  print(targetTime)
  
  # Bound the 'worst' performance to be five times the target time (according to PERF_QUOTIENT_NORMALISATION_COEFF)
  perfQuotient = (min(targetTime - timeTaken, targetTime*PERF_QUOTIENT_NORMALISATION_COEFF))/targetTime
  return perfQuotient

In [7]:
data = np.genfromtxt('data.csv', delimiter=',')

In [38]:
data = data.reshape(100000, 10, 1)
x = data[:, :-1]
y = data[:,-1]

train_x, train_y = x[:60000], y[:60000]
val_x, val_y = x[60000:80000], y[60000:80000]
test_x, test_y = x[80000:], y[80000:]

In [39]:
lstm_model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(1),
  tf.keras.layers.Dense(units=1)
])

lstm_model.compile(optimizer='sgd', loss='mse')
lstm_model.fit(x, y, batch_size=32, epochs=10)

Epoch 1/10
3125/3125 [==============================] - 8s 2ms/step - loss: 0.0954
Epoch 2/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0077
Epoch 3/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0050
Epoch 4/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0040
Epoch 5/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0035
Epoch 6/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0031
Epoch 7/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0029
Epoch 8/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0028
Epoch 9/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0026
Epoch 10/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0025


In [40]:
lstm_model.evaluate(test_x, test_y)

625/625 [==============================] - 1s 1ms/step - loss: 0.0025


0.00247415853664279